# 贝叶斯优化 #

机器学习方法中难以避免地要遇到一些超参数，例如数模型中的最大深度，神经网络中的学习率等等。 在特征和模型一定的情况下，参数的不同选择直接影响了算法的最终效果，对于具有大量超参数或者运行速度慢的模型，手动调参或者简单搜索是非常低效而且令人烦躁的。而贝叶斯优化方法被认为是一种有效的参数调优方法。  

## 简单的参数搜索调优方法 ##
首先介绍一下简单的参数搜索方法，网格搜索(Grid Search)是一种比较容易想出来的参数调优方法，我们为各个参数指定一个搜索空间，这样多个参数可以构建出一张搜索表，例如对于两个参数$\alpha和\beta$，限定它们的搜索空间都是为0-1，搜索步长均为0.1，那么就可以构成一张$10 * 10 $的搜索表，对于其中每一对参数组合，用验证集检验模型的性能，保存最佳的参数组合。
![](./grid.png)
网格搜索的效率很低，一旦参数数量较多，网格搜索的弊端就非常明显了，这种情况下随机搜索会更有效，随机搜索为每个参数设定一个分布，并在其中随机采样。
## 高斯过程先验的贝叶斯优化 ##
贝叶斯优化过程可以看做是在一个边界内寻找目标函数$f(x)$的最小值的过程。函数中的自变量x此时代表的是算法中的超参数。在我所了解过的目标函数中，大部分是易于优化的凸函数形式，然而当超参数作为自变量而不是已知量的时候，目标函数是未知的非凸函数，所以传统的梯度方法并不适用。
贝叶斯优化根据先验分布，假设采集函数而学习到目标函数的形状。在每一次使用新的采样点来测试目标函数时，它使用该信息来更新目标函数的先验分布。然后，算法测试由后验分布给出的最值可能点。
### 高斯过程 ###
高斯过程（Gaussian Process）是一个用来求出函数先验分布的方便而强大的方法。GP由以下性质定义：任何有限个点的集合$ \{{x_n \in\mathbb X} \}_{n=1}^{N}$ 在$R^N$空间上构成一个多维的高斯分布。第n个数据点被认为是函数值$f(x_n)$，我们可以结合高斯分布的边缘性质以闭合的形式计算出边际和条件。 多元高斯分布有两个决定性的参数：均值m和协方差K。GP过程的定义式表示如下：$$ {f(x)} \tilde{} {GP(m(x),K(x,x')}$$ 
在本文中，作者利用高斯过程获得$f(x)$的先验。
### 采集函数 ###
假设我们利用高斯过程先验得到了$f(x)$，我们可以观察到数据点的分布情况$\{X_n,y_n\}_{n=1}^{N}$,其中$ {y_n} \tilde{} {\rm N} \{f(x_n),v\}$,这里的v代表的是噪声引入的误差。 先验函数和这些数据点可以为我们引入下一步，求后验函数$a: {\rm X} \rightarrow R^+$。
实际上，我们真正想要求出的是下一个需要观测的采样点，因为贝叶斯优化实际上要解决的问题是找到那个使得函数值最小的那个x值，而我们已经观察到的哪些数据点可能并不是最优值。 下一个采样点将基于高斯过程的先验和最大化采集函数得到，所求得的下个采样点是我们此刻认为的能够使得目标函数有最小值的x。  
采集函数依赖于之前的观察点和GP中的超参数，所以我们把采集函数写成$a(x;\{x_n,y_n\},\theta)$的形式，我们把当前先验的观察点中使得函数$f$值最小的点记为$x_{best}$，标准正态分布函数记为$\phi(.)$，下面简单介绍几种常见的采集函数：
#### Probability of Improvement ####
这种采集函数的思想是最大化能够改进当前最优值的概率。$$a_{PI}(x;\{x_n,y_n\},\theta)=\phi( \gamma (x))$$,$$ \gamma(x)=\frac{f(X_{best})-\mu (x;\{x_n,y_n\},\theta)}{\sigma (x;\{x_n,y_n\},\theta)}$$
#### Expected Improvement ####
这种采集函数的思想是最大化比当前最优值的改进。$$a_{EI}(x;\{x_n,y_n\},\theta)=\sigma(x;\{x_n,y_n\},\theta)(\gamma(x)\phi(x))+{\rm N} (\gamma(x);0,1))$$
#### GP Upper Confidence Bound ####
还可以通过较低的置信边界来构造采集函数，使得最优化过程中的“regret”最小。此类采集的函数的形式如下：$$a_{LCB}(x;\{x_n,y_n\},\theta)=\mu(x;\{x_n,y_n\},\theta)-k\sigma(x;\{x_n,y_n\},\theta)$$,k用来平衡探索与开发。探测（exploration）就是在还未取样的区域获取采样点。开发（exploitation）就是根据后验分布，在最可能出现全局最优解的区域进行采样。我们下一个选取点（x）应该有比较大的均值（开发）和比较高的方差（探索）。
### 贝叶斯超参数优化的实际考虑 ###
上述介绍的贝叶斯优化方法有一些局限。首先，对于实际问题而言，协方差函数和与其相关的超参数的选择不明确；其次，由于函数评估本身可能涉及耗时的优化程序，所以问题可能在时间上显着变化，应该将其考虑在内。 第三，优化算法应该利用多核并行机制，以便很好地映射到现代计算环境。 在论文中，作者针对这些问题提出解决方案。
#### 协方差函数以及协方差超参数的处理 ####
自动相关确定(ARD）平方指数核函数是GP回归中常用的核函数：$$K_{SE}(x,x^{'})=\theta_0 exp\{-\frac{1}{2} r^2(x,x^{'})\} $$ $$ r^2(x,x^{'})=\sum_{d=1}^{D}{{(x_d-x^{'}_d)^2} / {\theta_d}^2}$$,在本文中，作者建议使用更具实用性的ARD Matern 5/2 kernel:$$K_{M52}(x,x^{'})=\theta_0 (1+\sqrt {5r^2(x,x^{'}})+\frac{5}{3} r^2(x,x^{'}) exp\{-\sqrt 5r^2(x,x^{'})\}$$
现在我们已经了协方差函数的形式，我们还需要处理其中涉及到的超参数，对于超参数（用θ表示）的全贝叶斯处理，希望边缘化超参数并计算综合采集函数:$$ \hat{a}(X;{x_n,y_n})=\int a（x;\{x_n,y_n\},\theta）p(\theta |\{x_n,y_n\}_{n=1}^{N})d\theta$$
![](./fig1.png)
上图中（a）显示了通过5个观测点获得的三个分布函数，(b)图显示了三条对应的EI采集函数，图(c)显示了这三条采集函数融合后的整合采集函数。
#### 采用蒙特卡洛采集函数的并行贝叶斯优化 ####
贝叶斯优化的目标是快速找到一组尽可能好的超参数，一个很自然的想法是进行并行优化。作者希望能够在一组数据点正在评估的同时，选择出下一个采样点。理想情况下，我们可以使用“转出”策略，选择适当平衡信息获取和利用的点。 但是，这种推出通常是棘手的。 相反，作者提出了一种序列化策略，它利用高斯过程的易处理的推理性质来计算蒙特卡罗估计的采集函数，这些估计函数可能来自待定函数评估的不同结果。

考虑N个评估已经完成的情况，产生数据$ \{x_n,y_n \}_{n=1}^{N}$ ，其中J个评估$\{X_j\}_{j=1}^{J}$仍在等待处理中。 理想情况下，我们将根据这些未决评估的所有可能结果，根据预期采集函数选择一个新点：
$$ \hat{a}(X;{x_n,y_n}，\theta,\{x_j\})=\int a（x;\{x_n,y_n\},\theta）p(\{y_j\}_{j=1}^{J}| \{x_j\}_{j=1}^{J},\{x_n,y_n\}_{n=1}^{N})dy_1 ……dy_J$$ 
![](./fig20.png)
上图显示了这一过程。



综合上述的介绍，贝叶斯优化的基本过程是通过高斯过程利用观察点获得f(x)的先验分布，再利用采样函数选择下一个采样点，将新的采样点加入观察点中重新进行高斯过程，循环迭代。
![](./fig3.png)
![](./Intuition0.png)
![](./Intuition1.png)

在作者实现的代码中，我们想要使用该贝叶斯优化方法只需要定义好返回的函数和参数的类型、取值范围即可,具体而言就是定义模型的函数，例如branin函数：



In [1]:
import numpy as np
import math

def branin(x, y):

    result = np.square(y - (5.1/(4*np.square(math.pi)))*np.square(x) + 
         (5/math.pi)*x - 6) + 10*(1-(1./(8*math.pi)))*np.cos(x) + 10
    
    result = float(result)
    
    print ('Result = %f' % result)
    #time.sleep(np.random.randint(60))
    return result


def main(job_id, params):
    print ('Anything printed here will end up in the output directory for job #%d' % job_id)
    print (params)
    return branin(params['x'], params['y'])


此外在config.json中定义采用的编程语言、模型文件、参数类型和取值等等：

In [2]:
{
    "language"        : "PYTHON",
    "main-file"       : "branin2.py",
    "experiment-name" : "simple2-braninhoo-example",
    "likelihood"      : "NOISELESS",
    "variables" : {
        "max_feature1" : {
            "type" : "FLOAT",
            "size" : 1,
            "min"  : 0,
            "max"  : 1
        },
        "max_feature2" : {
            "type" : "FLOAT",
            "size" : 1,
            "min"  : 0,
            "max"  : 1
        },
        "CC" : {
            "type" : "FLOAT",
            "size" : 1,
            "min"  : 0,
            "max"  : 1
        }
    }
}



{'experiment-name': 'simple2-braninhoo-example',
 'language': 'PYTHON',
 'likelihood': 'NOISELESS',
 'main-file': 'branin2.py',
 'variables': {'CC': {'max': 1, 'min': 0, 'size': 1, 'type': 'FLOAT'},
  'max_feature1': {'max': 1, 'min': 0, 'size': 1, 'type': 'FLOAT'},
  'max_feature2': {'max': 1, 'min': 0, 'size': 1, 'type': 'FLOAT'}}}

运行后我们可以得到推荐的参数取值：
![](./example2.png)